In [1]:
import sys
import os
import numpy as np 
from validation import compute_f1
from keras.models import Model
from keras.layers import TimeDistributed,Conv1D,Dense,Embedding,Input,Dropout,LSTM,Bidirectional,MaxPooling1D,Flatten,concatenate
from prepro import readfile,createBatches,createMatrices,iterate_minibatches,addCharInformatioin,padding
from keras.utils import plot_model,Progbar
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
from keras.optimizers import Adam
from sklearn.metrics import precision_recall_fscore_support as score
import sklearn
import subprocess

/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
epochs = 80
trainable = True

In [3]:
from keras_contrib.layers import CRF


In [4]:
def tag_dataset(dataset):
    correctLabels = []
    predLabels = []
    b = Progbar(len(dataset))
    for i,data in enumerate(dataset):    
        tokens, casing,char, labels = data
        tokens = np.asarray([tokens])     
        casing = np.asarray([casing])
        char = np.asarray([char])
        pred = model.predict([tokens, casing,char], verbose=False)[0]   
        pred = pred.argmax(axis=-1) #Predict the classes            
        correctLabels.append(labels)
        predLabels.append(pred)
        b.update(i)
    return predLabels, correctLabels

In [5]:
# changing all deriv and part to misc. with BIO
def modify_labels(dataset):
    bad_labels = ['I-PERderiv','I-OTHpart','B-ORGderiv', 'I-OTH','B-OTHpart','B-LOCderiv','I-LOCderiv','I-OTHderiv','B-PERderiv','B-OTHderiv','B-PERpart','I-PERpart','I-LOCpart','B-LOCpart','I-ORGpart','I-ORGderiv','B-ORGpart','B-OTH']
    for sentence in dataset:
        for word in sentence:
            label = word[1]
            if label in bad_labels:
                first_char = label[0]
                if first_char == 'B' :
                    word[1] = 'B-MISC'
                else:
                    word[1] = 'I-MISC'
    return dataset
                

In [57]:

# preproecessing data from GermEval
def get_sentences(path):
    sentences=[]
    with open(path,'rb') as f:
    #     lines = f.readlines()
        sentence=[]
        for line in f:
            try:
                splits = [x.decode() for x in line.split()]
            except Exception as e:
                continue
            try:
                if splits[0]!='#':
                    temp = [splits[1],splits[2]]
                    sentence.append(temp)
                else:
                    if len(sentence):
                        sentences.append(sentence)
                    sentence=[]
            except Exception as e:
                continue
    return sentences



In [58]:
train_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-train.tsv')
dev_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-dev.tsv')
test_sentences = get_sentences('/home/jindal/notebooks/Resources/GermEVAL/NER-de-test.tsv')

print(len(train_sentences))
print(len(dev_sentences))
print(len(test_sentences))


23999
2199
5099


In [59]:
print(train_sentences[0])

[['Schartau', 'B-PER'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'B-ORG'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'B-PER'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [60]:
fast_text_words=[]

with open('/home/jindal/notebooks/fastText/wiki.de.vec','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
        fast_text_words.append(word)

In [61]:
del fast_text_words[0]
print(fast_text_words[0])

.


In [62]:
print(train_sentences[100])

[['Der', 'O'], ['Meteoritensucher', 'O'], ['hatte', 'O'], ['im', 'O'], ['Frühjahr', 'O'], ['auf', 'O'], ['der', 'O'], ['dänischen', 'B-LOCderiv'], ['Insel', 'O'], ['Lolland', 'B-LOC'], ['30', 'O'], ['Gramm', 'O'], ['eines', 'O'], ['Meteoriten', 'O'], ['aufgespürt', 'O'], [',', 'O'], ['der', 'O'], ['Mitte', 'O'], ['Januar', 'O'], ['über', 'O'], ['Nordeuropa', 'B-LOC'], ['beobachtet', 'O'], ['worden', 'O'], ['war', 'O'], ['.', 'O']]


In [63]:
# train_sentences = modify_labels(train_sentences)
# dev_sentences = modify_labels(dev_sentences)
# test_sentences = modify_labels(test_sentences)

In [64]:
trainSentences = addCharInformatioin(train_sentences)
devSentences = addCharInformatioin(dev_sentences)
testSentences = addCharInformatioin(test_sentences)


In [65]:
test_sentences_in_fasttext=[]
test_sentences_not_in_fasttext=[]

for sentence in testSentences:
    flag=False
    for word in sentence:
        token=word[0].lower()
#         print(token)
        if token not in fast_text_words:
            flag=True
            break
    if flag:
        test_sentences_not_in_fasttext.append(sentence)
    else:
        test_sentences_in_fasttext.append(sentence)

In [66]:
print(len(test_sentences_in_fasttext))
print(len(test_sentences_not_in_fasttext))
print(len(test_sentences))

2204
2895
5099


In [67]:
labelSet = set()
words = {}
characters= set()

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for token, char, label in sentence:
            for c in char:
                characters.add(c)
            labelSet.add(label)
            words[token.lower()] = True
            # words[token] = True

In [68]:
print(labelSet)

{'B-OTHderiv', 'B-LOCpart', 'I-ORGderiv', 'B-OTH', 'B-LOCderiv', 'B-ORGpart', 'B-ORG', 'B-ORGderiv', 'I-OTHderiv', 'I-ORG', 'I-PERpart', 'I-PER', 'I-ORGpart', 'B-PERderiv', 'B-LOC', 'I-PERderiv', 'I-OTHpart', 'I-LOCderiv', 'B-OTHpart', 'I-LOCpart', 'I-LOC', 'O', 'B-PER', 'B-PERpart', 'I-OTH'}


In [69]:
# :: Create a mapping for the labels ::
label2Idx = {}
for label in labelSet:
    label2Idx[label] = len(label2Idx)

In [70]:
print(label2Idx)

{'B-ORGpart': 5, 'B-ORG': 6, 'B-OTHderiv': 0, 'B-LOCpart': 1, 'B-ORGderiv': 7, 'I-OTHderiv': 8, 'I-ORG': 9, 'I-ORGderiv': 2, 'I-PERpart': 10, 'I-PER': 11, 'I-ORGpart': 12, 'B-PERderiv': 13, 'B-LOC': 14, 'B-OTH': 3, 'I-OTHpart': 16, 'I-LOCderiv': 17, 'I-PERderiv': 15, 'B-LOCderiv': 4, 'B-OTHpart': 18, 'I-LOCpart': 19, 'I-LOC': 20, 'O': 21, 'B-PER': 22, 'B-PERpart': 23, 'I-OTH': 24}


In [71]:
# :: Hard coded case lookup ::
case2Idx = {'numeric': 0, 'allLower':1, 'allUpper':2, 'initialUpper':3, 'other':4, 'mainly_numeric':5, 'contains_digit': 6, 'PADDING_TOKEN':7}
caseEmbeddings = np.identity(len(case2Idx), dtype='float32')

In [72]:
string_words = ' '.join(words.keys())
# print(string_words)
f = open('/home/jindal/notebooks/fastText/german_words.txt','wb')
f.write(string_words.encode())
f.close()

In [73]:
print(caseEmbeddings)
print(case2Idx)

[[1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1.]]
{'mainly_numeric': 5, 'allLower': 1, 'contains_digit': 6, 'PADDING_TOKEN': 7, 'allUpper': 2, 'initialUpper': 3, 'other': 4, 'numeric': 0}


In [74]:
word2Idx={}
wordEmbeddings=[]

# created a file by the name of german_words.txt in /fastText. Containing all the words in our dataset
with open('german_word_embeddings.txt','rb') as f:
    for line in f:
        splits = line.split()
        word = splits[0].decode()
#         print(word.decode())
        if len(word2Idx) == 0: #Add padding+unknown
            word2Idx["PADDING_TOKEN"] = len(word2Idx)
            vector = np.zeros(len(splits)-1) #Zero vector vor 'PADDING' word
            wordEmbeddings.append(vector)

            word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
            vector = np.random.uniform(-0.25, 0.25, len(splits)-1)
            wordEmbeddings.append(vector)
            
        word2Idx[word.lower()]=len(word2Idx)
        embedding = np.array([float(num) for num in splits[1:]])
        wordEmbeddings.append(embedding)
wordEmbeddings=np.array(wordEmbeddings)

In [75]:
# print(wordEmbeddings[2])
print(len(wordEmbeddings))
print(len(wordEmbeddings[2]))

86688
300


In [76]:
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)
print(char2Idx)

{'ó': 0, 'ř': 1, 'к': 216, '柯': 3, 'С': 5, 'γ': 273, 'ệ': 8, 'ю': 274, '“': 9, '별': 220, 'ς': 10, '→': 11, '\x92': 12, 'в': 13, 'п': 15, '博': 16, 'Þ': 21, 'ê': 53, 'о': 20, 'η': 22, 'λ': 23, '‐': 2, 'N': 189, 'τ': 24, 'ĩ': 55, '別': 26, 'ś': 27, 'ъ': 28, 'o': 29, '寝': 31, '$': 6, 'В': 32, 'Ł': 33, '[': 35, '×': 36, 'ö': 37, '鷹': 38, 'p': 39, '³': 40, '術': 42, 'β': 43, 'å': 44, '&': 318, '동': 45, 'Ö': 116, 'ό': 46, 'í': 47, 'έ': 48, 'F': 49, '貴': 323, 'á': 50, '−': 51, 'є': 52, 'B': 167, '章': 54, 'ñ': 56, 'j': 57, 'Ş': 229, 'Ш': 59, '£': 60, '″': 62, 'H': 63, 'z': 64, 'у': 61, '<': 179, 'e': 67, 'д': 282, '‹': 69, 'Č': 70, 'ô': 71, '⊃': 80, 'ŏ': 77, 'ú': 78, 'ł': 79, '樓': 81, '九': 83, '#': 176, '鶴': 84, '5': 85, ':': 65, '3': 86, 'W': 87, '8': 89, 'À': 90, 'V': 92, 'ي': 94, 'A': 68, 'y': 251, 'g': 95, 'ň': 97, 'Л': 100, '+': 98, '6': 99, 'µ': 285, 'ő': 91, 'ض': 101, 'с': 102, 'ē': 197, 'ð': 74, 'ế': 104, 'è': 105, '佐': 106, 'È': 107, 'ć': 18, 'k': 108, 'У': 110, 'ť': 111, 'õ': 237, '公': 

In [77]:
print(trainSentences[0]) 

[['Schartau', ['S', 'c', 'h', 'a', 'r', 't', 'a', 'u'], 'B-PER'], ['sagte', ['s', 'a', 'g', 't', 'e'], 'O'], ['dem', ['d', 'e', 'm'], 'O'], ['"', ['"'], 'O'], ['Tagesspiegel', ['T', 'a', 'g', 'e', 's', 's', 'p', 'i', 'e', 'g', 'e', 'l'], 'B-ORG'], ['"', ['"'], 'O'], ['vom', ['v', 'o', 'm'], 'O'], ['Freitag', ['F', 'r', 'e', 'i', 't', 'a', 'g'], 'O'], [',', [','], 'O'], ['Fischer', ['F', 'i', 's', 'c', 'h', 'e', 'r'], 'B-PER'], ['sei', ['s', 'e', 'i'], 'O'], ['"', ['"'], 'O'], ['in', ['i', 'n'], 'O'], ['einer', ['e', 'i', 'n', 'e', 'r'], 'O'], ['Weise', ['W', 'e', 'i', 's', 'e'], 'O'], ['aufgetreten', ['a', 'u', 'f', 'g', 'e', 't', 'r', 'e', 't', 'e', 'n'], 'O'], [',', [','], 'O'], ['die', ['d', 'i', 'e'], 'O'], ['alles', ['a', 'l', 'l', 'e', 's'], 'O'], ['andere', ['a', 'n', 'd', 'e', 'r', 'e'], 'O'], ['als', ['a', 'l', 's'], 'O'], ['überzeugend', ['ü', 'b', 'e', 'r', 'z', 'e', 'u', 'g', 'e', 'n', 'd'], 'O'], ['war', ['w', 'a', 'r'], 'O'], ['"', ['"'], 'O'], ['.', ['.'], 'O']]


In [78]:
# createMatrices: for every sentence, changes its words, cases,characters, labels to its corresponding id in their embeddings
# padding is used to pad the character indices to a fixed size=52
train_set = padding(createMatrices(trainSentences, word2Idx,  label2Idx, case2Idx, char2Idx))
dev_set = padding(createMatrices(devSentences, word2Idx, label2Idx, case2Idx, char2Idx))
test_set = padding(createMatrices(testSentences, word2Idx, label2Idx, case2Idx, char2Idx))
test_set_in_fasttext= padding(createMatrices(test_sentences_in_fasttext, word2Idx, label2Idx, case2Idx, char2Idx))
test_set_not_in_fasttext = padding(createMatrices(test_sentences_not_in_fasttext, word2Idx, label2Idx, case2Idx, char2Idx))

In [79]:
# train-set[][0]: corresponds to the ids of the words in the sentence
# train_set[][1]: corresponds to the ids of the cases of the words
# train_set[][2]: contains numpy arrays, one corresponding to every word, each containing the indices of the characters of that word
# the numpy arrays have a fixed size (padding or truncation) to 52
# train_set[][3]: corresponds to the ids of the labels of every word

print(train_set[0])
print(len(train_set[0][0])) # gives the number of words in the sentence
print((train_set[0][2]))

[[66621, 48489, 64378, 48605, 45132, 48605, 71994, 24199, 9866, 61086, 49918, 48605, 56689, 56275, 81412, 41725, 9866, 53656, 77940, 59258, 86062, 76881, 22538, 48605, 83973], [3, 1, 1, 4, 3, 4, 1, 3, 4, 3, 1, 4, 1, 1, 3, 1, 4, 1, 1, 1, 1, 1, 1, 4, 4], array([[113, 255, 132, ...,   0,   0,   0],
       [188,  17,  95, ...,   0,   0,   0],
       [ 96,  67, 171, ...,   0,   0,   0],
       ...,
       [284,  17, 316, ...,   0,   0,   0],
       [259,   0,   0, ...,   0,   0,   0],
       [253,   0,   0, ...,   0,   0,   0]], dtype=int32), [22, 21, 21, 21, 6, 21, 21, 21, 21, 22, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21]]
25
[[113 255 132 ...   0   0   0]
 [188  17  95 ...   0   0   0]
 [ 96  67 171 ...   0   0   0]
 ...
 [284  17 316 ...   0   0   0]
 [259   0   0 ...   0   0   0]
 [253   0   0 ...   0   0   0]]


In [80]:
idx2Label = {v: k for k, v in label2Idx.items()}

train_batch,train_batch_len = createBatches(train_set)
dev_batch,dev_batch_len = createBatches(dev_set)
test_batch,test_batch_len = createBatches(test_set)

In [81]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=0


In [82]:
print(test_sentences_in_fasttext[0])

[['Die', ['D', 'i', 'e'], 'O'], ['These', ['T', 'h', 'e', 's', 'e'], 'O'], [',', [','], 'O'], ['Schlatter', ['S', 'c', 'h', 'l', 'a', 't', 't', 'e', 'r'], 'B-PER'], ['sei', ['s', 'e', 'i'], 'O'], ['Antisemit', ['A', 'n', 't', 'i', 's', 'e', 'm', 'i', 't'], 'O'], ['gewesen', ['g', 'e', 'w', 'e', 's', 'e', 'n'], 'O'], [',', [','], 'O'], ['wurde', ['w', 'u', 'r', 'd', 'e'], 'O'], ['seither', ['s', 'e', 'i', 't', 'h', 'e', 'r'], 'O'], ['in', ['i', 'n'], 'O'], ['der', ['d', 'e', 'r'], 'O'], ['theologischen', ['t', 'h', 'e', 'o', 'l', 'o', 'g', 'i', 's', 'c', 'h', 'e', 'n'], 'O'], ['Fachliteratur', ['F', 'a', 'c', 'h', 'l', 'i', 't', 'e', 'r', 'a', 't', 'u', 'r'], 'O'], ['nicht', ['n', 'i', 'c', 'h', 't'], 'O'], ['mehr', ['m', 'e', 'h', 'r'], 'O'], ['vertreten', ['v', 'e', 'r', 't', 'r', 'e', 't', 'e', 'n'], 'O'], ['.', ['.'], 'O']]


In [83]:
words_input = Input(shape=(None,),dtype='int32',name='words_input')
words = Embedding(input_dim=wordEmbeddings.shape[0], output_dim=wordEmbeddings.shape[1],  weights=[wordEmbeddings], trainable=False)(words_input)
casing_input = Input(shape=(None,), dtype='int32', name='casing_input')
casing = Embedding(output_dim=caseEmbeddings.shape[1], input_dim=caseEmbeddings.shape[0], weights=[caseEmbeddings], trainable=False)(casing_input)
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),32,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
# dropout= Dropout(0.5, name='dropout1')(embed_char_out)

# conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
# maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
# char = TimeDistributed(Flatten())(maxpool_out)
# char = Dropout(0.5)(char)
# output = concatenate([words, casing, char])

kernel_sizes = (3, 4, 5)
conv_blocks = []
for sz in kernel_sizes:
    conv = TimeDistributed(Conv1D(
                         kernel_size=sz,
                         filters=32,
                         padding="same",
                         activation="relu",
                         strides=1))(embed_char_out)
    conv = TimeDistributed(MaxPooling1D(52))(conv)
    conv = TimeDistributed(Flatten())(conv)
#     conv = Dropout(0.5)(conv)
    conv_blocks.append(conv)
output = concatenate([words, casing, conv_blocks[0], conv_blocks[1], conv_blocks[2]])

output = Bidirectional(LSTM(200, return_sequences=True, dropout=0.50, recurrent_dropout=0.5))(output)
output = TimeDistributed(Dense(len(label2Idx)))(output)
crf = CRF(len(label2Idx))
output = crf(output)
model = Model(inputs=[words_input, casing_input,character_input], outputs=[output])
# model.compile(loss=crf.loss_function, optimizer=Adam(lr = 0.002), metrics=[crf.accuracy])
model.compile(loss=crf.loss_function, optimizer='nadam', metrics=[crf.accuracy])
model.summary()
# plot_model(model, to_file='model.png')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10496       char_input[0][0]                 
__________________________________________________________________________________________________
time_distributed_21 (TimeDistri (None, None, 52, 32) 3104        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distributed_24 (TimeDistri (None, None, 52, 32) 4128        char_embedding[0][0]             
__________________________________________________________________________________________________
time_distr

In [85]:
for x in range(1):
#     print(i)
    maxf1 = 0
    for epoch in range(epochs):    
        print("Epoch %d/%d"%(epoch, epochs))
        a = Progbar(len(train_batch_len))
        for i, batch in enumerate(iterate_minibatches(train_batch, train_batch_len)):
            labels, tokens, casing, char = batch
            labels = labels.tolist()
    #         print(labels)
            for sentence in labels:
                for i,label in enumerate(sentence):
                    temp = [0]*len(label2Idx)
                    value = label[0]
                    temp[value]=1
                    sentence[i] = temp
            labels = np.array(labels)
    #         print(labels)
            model.train_on_batch([tokens, casing, char], labels)
            a.update(i)
        if epoch >= 50:
            predLabels, correctLabels = tag_dataset(dev_batch)        
            pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
#             predLabels = np.concatenate(predLabels).ravel()
#             correctLabels = np.concatenate(correctLabels).ravel()
#             f1_dev = sklearn.metrics.f1_score(correctLabels,predLabels,average='macro' )
            print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))
#             print(f1_dev)

            if f1_dev > maxf1:
                model.save('german_ner_oov.h5')
                maxf1 = f1_dev

    id2word = {v: k for k, v in word2Idx.items()}
    print(len(correctLabels))        

    

Epoch 0/80
55/51 [================================] - 14s 254ms/step
Epoch 1/80
55/51 [================================] - 13s 237ms/step
Epoch 2/80
55/51 [================================] - 14s 253ms/step
Epoch 3/80
55/51 [================================] - 13s 236ms/step
Epoch 4/80
55/51 [================================] - 13s 242ms/step
Epoch 5/80
55/51 [================================] - 13s 240ms/step
Epoch 6/80
55/51 [================================] - 14s 248ms/step
Epoch 7/80
55/51 [================================] - 13s 233ms/step
Epoch 8/80
55/51 [================================] - 14s 252ms/step
Epoch 9/80
55/51 [================================] - 13s 228ms/step
Epoch 10/80
55/51 [================================] - 14s 256ms/step
Epoch 11/80
55/51 [================================] - 13s 236ms/step
Epoch 12/80
55/51 [================================] - 14s 252ms/step
Epoch 13/80
55/51 [================================] - 13s 228ms/step
Epoch 14/80
55/51 [===========

2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.817, Rec: 0.823, F1: 0.820
Epoch 59/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.824, Rec: 0.831, F1: 0.827
Epoch 60/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.831, Rec: 0.820, F1: 0.825
Epoch 61/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.830, Rec: 0.825, F1: 0.828
Epoch 62/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.816, Rec: 0.825, F1: 0.821
Epoch 63/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.817, Rec: 0.825, F1: 0.821
Epoch 64/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.821, Rec: 0.825, F1: 0.823
Epoch 65/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.819, Rec: 0.828, F1: 0.823
Epoch 66/80
2198/2199 [============================>.] - ETA: 0sDev-Data: Prec: 0.825, Rec: 0.831, F1: 0.828
Epoch 67/80
2198/2199 [========

In [88]:
model.load_weights('german_ner_oov.h5')
test_batch_not_in_fasttext, y = createBatches(test_set_not_in_fasttext)
predLabels, correctLabels = tag_dataset(test_batch_not_in_fasttext)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

id = 1
file_name='test_oov_fasttext'+str(x)+'.txt'
input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
f = open(input_file_name,'wb')
for sentence_number, batch in enumerate(test_batch_not_in_fasttext):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
#             string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
        string = string.encode()
        f.write(string)
        id+=1

command = "perl nereval.perl"
directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
#     directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
#     command = "perl conlleval"

output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_oov_fasttext'+str(x)+'.txt'
output_file=open(output_file_name,'w')
input_file = open(input_file_name)
import subprocess
process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_germeval)
out, err = process.communicate()
#     print(out)
print('******************************************************************************')

2894/2895 [============================>.] - ETA: 0sDev-Data: Prec: 0.806, Rec: 0.774, F1: 0.790
******************************************************************************


In [90]:
model.load_weights('german_ner_oov.h5')
test_batch_in_fasttext , x =  createBatches(test_set_in_fasttext)
predLabels, correctLabels = tag_dataset(test_batch_in_fasttext)        
pre_dev, rec_dev, f1_dev = compute_f1(predLabels, correctLabels, idx2Label)
print("Dev-Data: Prec: %.3f, Rec: %.3f, F1: %.3f" % (pre_dev, rec_dev, f1_dev))

id = 1
file_name='test_iv_fasttext'+str(x)+'.txt'
input_file_name = '/home/jindal/notebooks/jindal/NER/'+file_name
f = open(input_file_name,'wb')
for sentence_number, batch in enumerate(test_batch_in_fasttext):
    for word_number, wordid in enumerate(batch[0]):
        word = id2word[wordid]
        true_label = correctLabels[sentence_number][word_number]
        true_label = idx2Label[true_label]
        pred_label = predLabels[sentence_number][word_number]
        pred_label = idx2Label[pred_label]

        string = str(id) + '\t' + word + '\t' + true_label +'\t' + true_label+'\t'+pred_label+'\t'+pred_label+'\t\n'
#             string = word + ' ' + pred_label+' '+pred_label+' '+true_label+'\n'
        string = string.encode()
        f.write(string)
        id+=1

command = "perl nereval.perl"
directory_germeval = r'/home/jindal/notebooks/Resources/GermEVAL'
#     directory_conll = r'/home/jindal/notebooks/Resources/CONLL/2003/ner/bin'
#     command = "perl conlleval"

output_file_name = '/home/jindal/notebooks/jindal/NER/test_result_iv_ft'+str(x)+'.txt'
output_file=open(output_file_name,'w')
input_file = open(input_file_name)
import subprocess
process = subprocess.Popen(command.split(), stdin=input_file, stdout=output_file, cwd=directory_germeval)
out, err = process.communicate()
#     print(out)
print('******************************************************************************')

2203/2204 [============================>.] - ETA: 0sDev-Data: Prec: 0.871, Rec: 0.839, F1: 0.855
******************************************************************************
